## Requête 1: liste des acteurs par film


In [2]:
# import des librairies
import os
import pandas as pd 
import numpy as np

In [32]:

# lire les fichiers .tsv vers un dataframe
df_name_basics = pd.read_csv('../name.basics.tsv', sep="\t", nrows = 80000, usecols=["nconst", "primaryName"])

#df_name_basics = df_name_basics[["nconst", "primaryName", "knownForTitles"]]
#
df_principals = pd.read_csv('../title.principals.tsv', sep="\t", nrows = 10000) 
#
df_principals = df_principals.loc[(df_principals['category']=="actor")|(df_principals['category']=="actress"),:]
df_principals = df_principals[["tconst", "nconst"]]
# inner join with merge
df_merged_inner = pd.merge(left = df_principals, right= df_name_basics, left_on='nconst', right_on='nconst')
#lire title.basics


In [12]:
# lire le fichier title.basics.tsv
df_title_basics = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 80000)
# mettre un filtre sur titleType
df_title_basics = df_title_basics.loc[(df_title_basics['titleType']=="tvMovie")|(df_title_basics['titleType']=="movie"),:]
# select interesting columns
df_title_filtred = df_title_basics[["tconst", "primaryTitle"]] 
#faire un merge
liste_actor = pd.merge(left=df_title_filtred, right = df_merged_inner, left_on = 'tconst', right_on = 'tconst', how = 'inner')
liste_actor = liste_actor[["primaryTitle","primaryName"]]
#
#liste_actor.to_csv('liste_actor.csv', sep =",")
#groupby
liste_actors = liste_actor.groupby(['primaryTitle'])['primaryName'].apply(','.join).reset_index()
liste_actors.to_csv('liste_actors.csv', sep =",")
#print(liste_actors.head())

## Requête 2: liste des films américains avec leurs titres en français et leur moyenne

In [20]:
# lire le fichier.tsv
df_title_akas = pd.read_csv('../title.akas.tsv', sep="\t", nrows = 80000, usecols=["titleId", "title", "region"])
#
df_title_akas_US = df_title_akas.loc[(df_title_akas['region']=="US"),:]
#
df_title_akas_US = df_title_akas_US.rename(columns={'region':'region_US'})
#
df_title_akas_FR = df_title_akas.loc[(df_title_akas['region']=="FR"),:]
#
df_title_akas_FR = df_title_akas_FR.rename(columns={'title': 'title_french','region':'region_FR'})
#merge des deux fichiers df_title_akas_US et df_title_akas_FR
df_title_filtred_akas = pd.merge(left= df_title_akas_US, right = df_title_akas_FR, left_on = 'titleId', right_on = 'titleId', how = 'left')
# selection des colonnes intéressantes
df_title_filtred_akas = df_title_filtred_akas[["titleId", "title_french", "title"]]
# nettoyage du dataframe df_title_filtred
df_title_filtred_akas['title'] = np.where(df_title_filtred_akas['title_french'].notnull(), '', df_title_filtred_akas['title'])
#
df_title_filtred_akas = df_title_filtred_akas.rename(columns ={'titleId': 'tconst'})
# faire un merge avec la liste des films filtrés (movie et intermvie)
df_title_filtred = df_title_basics[["tconst"]] 
#
df_title_filtred_akas =pd.merge(left = df_title_filtred, right = df_title_filtred_akas, left_on = 'tconst', right_on = 'tconst', how = 'inner')



In [22]:
df_title_rating = pd.read_csv('../title.ratings.tsv', sep="\t", nrows = 100000, usecols=["tconst", "averageRating"])
# merge des deux fichiers df_title_rating et df_title_filtred_akas
liste_film_americain = pd.merge(left = df_title_filtred_akas, right = df_title_rating, left_on = 'tconst', right_on = 'tconst', how = 'inner')


## Requête 3: les notes moyennes des différents genres

In [31]:
df_title_basics_2 = df_title_basics[["tconst","genres"]] 
#
df_title_basics_2 = pd.DataFrame(df_title_basics_2.genres.str.split(',').tolist(), index=df_title_basics_2.tconst).stack()
df_title_basics_2 =df_title_basics_2.reset_index([0, 'tconst'])
df_title_basics_2.columns = ['tconst', 'genres']
# récupération des notes par un merge
note_genre = pd.merge(left = df_title_basics_2, right = df_title_rating, left_on = 'tconst', right_on = 'tconst', how = 'inner')



## Requête 4: la note moyenne de chaque acteur par rapport aux films dans lesquels il apparait

In [35]:
#merge des fichiers df_merged_inner et df_title_rating
note_actor = pd.merge(left = df_merged_inner, right = df_title_rating, left_on = 'tconst', right_on = 'tconst', how = 'inner')
note_actor = note_actor[["primaryName", "averageRating"]]
# 
moyenne_actor = note_actor.groupby('primaryName')['averageRating'].mean()

In [37]:
df4 = pd.read_csv('../title.ratings.tsv', sep="\t", nrows = 1000)
print(df4.head())

tconst  averageRating  numVotes
0  tt0000001            5.6      1550
1  tt0000002            6.1       186
2  tt0000003            6.5      1207
3  tt0000004            6.2       113
4  tt0000005            6.1      1934


In [13]:
df1 = pd.read_csv('../title.basics.tsv', sep="\t", nrows = 500)
print(df1.head())

      tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   
2  tt0000003     short          Pauvre Pierrot          Pauvre Pierrot   
3  tt0000004     short             Un bon bock             Un bon bock   
4  tt0000005     short        Blacksmith Scene        Blacksmith Scene   

   isAdult  startYear endYear runtimeMinutes                    genres  
0        0       1894      \N              1         Documentary,Short  
1        0       1892      \N              5           Animation,Short  
2        0       1892      \N              4  Animation,Comedy,Romance  
3        0       1892      \N             \N           Animation,Short  
4        0       1893      \N              1              Comedy,Short  


In [24]:
del(df_title_akas)